In [ ]:
import json
import pandas as pd
import plotly.express as px

In [ ]:
df_2019 = pd.read_csv("data/broadband_data_2019November.csv")
df_2019 = df_2019.query("ST=='NJ'")

In [ ]:

with open("shapefiles/geojson-counties-fips.json") as f:
    counties = json.load(f)

In [ ]:
def get_numbers(str_data):
    return float(str_data.strip())


df_2019['broadband_usage'] = df_2019[' BROADBAND USAGE '].apply(get_numbers)
df_2019['broadband_availability'] = df_2019[' BROADBAND AVAILABILITY PER FCC '].apply(get_numbers)
df_2019


In [ ]:
fig = px.choropleth(df_2019, geojson=counties, locations='COUNTY ID', color='broadband_usage',
                           color_continuous_scale="armyrose_r",
                           range_color=(0, 1),
                           scope="usa",
                           labels={'broadband_usage':'usage'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.add_scattergeo(
                   locations = df_2019['COUNTY ID'],
                   text = df_2019['broadband_usage'],
                   geojson=counties,
                   mode = 'text',
                   )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth(df_2019, geojson=counties, locations='COUNTY ID', color='broadband_availability',
                           color_continuous_scale="armyrose_r",
                           range_color=(0, 1),
                           scope="usa",
                           labels={'broadband_availability':'availability'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
df_2019

In [ ]:
with open("shapefiles/nj_new_jersey_zip_codes_geo.min.json") as f:
    zipcodes = json.load(f)

In [ ]:
bb_zip_df = pd.read_csv("data/broadband_data_zipcode.csv")
bb_zip_df = bb_zip_df.query("ST=='NJ'")[["POSTAL CODE", "BROADBAND USAGE"]]
bb_zip_df["POSTAL CODE"] = bb_zip_df["POSTAL CODE"].apply(str)
bb_zip_df["POSTAL CODE"] = bb_zip_df["POSTAL CODE"].str.pad(5, "left", "0")


In [ ]:
bb_zip_df

In [ ]:
zipcodes

In [ ]:
fig = px.choropleth(bb_zip_df, geojson=zipcodes, locations='POSTAL CODE', color='BROADBAND USAGE',
                           color_continuous_scale="armyrose_r",
                           range_color=(0, 1),
                           featureidkey="properties.ZCTA5CE10",
                           scope="usa",
                           labels={'BROADBAND USAGE':'usage'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()